***Authors : Akeniouene & Boutaleb & Lakhdar***

In [186]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing


In [187]:
nb_nodes= 500
G = nx.DiGraph(nx.barabasi_albert_graph(nb_nodes,9))
G.number_of_edges()

8838

In [188]:

rands = np.random.choice(500, 100, replace=False)
G.remove_edges_from(np.array(G.edges)[rands])

In [189]:
G.number_of_edges()

8738

In [190]:
#nx.draw_networkx(G, with_labels=True)

get the eigen values and eigen vectors

In [191]:

def transform(G,df=0.85):
    A = nx.to_numpy_array(G).T
    D = np.diag(1/np.sum(A, axis=0))
    M = A@D 
    M = df*M + (1-df)*np.ones((nb_nodes,nb_nodes))/nb_nodes
    return M


In [192]:
def normalisation(x):
    return np.abs(x).sum()

In [193]:
def pagerank_(G, df=0.85):
    M = transform(G, df=df)
    eigenvalues, eigenvectors = np.linalg.eig(M)
    idx = eigenvalues.argsort()[-1]
    largest = np.array(eigenvectors[:,idx]).flatten().real
    return largest / normalisation(largest)

On compare avec les résuktats de `pagerank` de `networkx`

In [194]:
pr = nx.pagerank(G,alpha=.85)

In [195]:
np.array([val for i,val in enumerate(pr.values()) if i < 10])

array([0.01121454, 0.00302886, 0.0038873 , 0.00737116, 0.00519186,
       0.00655222, 0.00159468, 0.00347869, 0.00804447, 0.00546427])

In [196]:
pagerank_(G)[:10]

array([0.01121471, 0.00302925, 0.00388736, 0.00737099, 0.00519223,
       0.00655193, 0.00159456, 0.00347823, 0.00804485, 0.00546418])

La norme vaut bien 1.

In [197]:
np.linalg.norm(pagerank_(G),1)

0.9999999999999998

Sur un dataset Réel :

In [225]:
data = pd.read_csv('ionosphere.data')
X_real = data.iloc[:,:-1]
y_real = data.iloc[:,-1]
y_real = pd.factorize(y_real)[0] #To transform y values to a categorial type
y_real.shape,X_real.shape

((350,), (350, 34))

In [226]:
nb_nodes= 350

In [227]:
X_real = X_real.to_numpy()

In [228]:
from sklearn.neighbors import kneighbors_graph 

def knn(X):
    connectivity = kneighbors_graph(
        X, 15, include_self=False
    )
    W = connectivity.toarray()
    L = np.diag(W.sum(axis=1)) - W
    return W,L

In [229]:
W_real_knn = knn(X_real)[0]

In [230]:
W_real_knn.shape

(350, 350)

In [231]:
G1 = nx.DiGraph(W_real_knn)

In [232]:
G1.number_of_edges()

5250

In [233]:
pagerank_(G1)[:10]

array([0.0005045 , 0.00122552, 0.0004557 , 0.00086382, 0.00843596,
       0.00114415, 0.00047868, 0.00135411, 0.00397399, 0.0012592 ])

In [234]:
np.array([val for i,val in enumerate(nx.pagerank(G1).values()) if i < 10])

array([0.0005045 , 0.00122581, 0.0004557 , 0.00086382, 0.00844313,
       0.00114415, 0.00047868, 0.00135411, 0.00397737, 0.0012592 ])

In [235]:
np.linalg.norm(pagerank_(G1),1)

1.0